<a href="https://colab.research.google.com/github/JazmineOrtizMarin/Simulaci-n-2/blob/main/MC_Antit%C3%A9ticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import time
from random import random

In [23]:
def g(x):
  return np.sqrt(np.arctan(x))

In [24]:
n = 300
repeticiones=50

##**Método Crudo**

In [25]:
def crudo_antitetico(n):
    suma_r = 0
    suma_1r = 0
    for i in range(n):
        r = rd.random()
        suma_r += g(r)          # theta1(r)
        suma_1r += g(1-r)       # theta1(1-r)
    # Promedios individuales y combinado
    I1 = suma_r / n
    I2 = suma_1r / n
    I12 = (I1 + I2) / 2
    return I1, I2, I12

In [26]:
def evaluar_antitetico(n, repeticiones):
    inicio_tiempo = time.time()

    lista_I1 = []
    lista_I2 = []
    lista_I12 = []

    for _ in range(repeticiones):
        i1, i2, i12 = crudo_antitetico(n)
        lista_I1.append(i1)
        lista_I2.append(i2)
        lista_I12.append(i12)

    fin_tiempo = time.time()
    tiempo_total = fin_tiempo - inicio_tiempo

    # Medias y varianzas de cada caso
    mu_I1, var_I1 = np.mean(lista_I1), np.var(lista_I1)
    mu_I2, var_I2 = np.mean(lista_I2), np.var(lista_I2)
    mu_I12, var_I12 = np.mean(lista_I12), np.var(lista_I12)

    return tiempo_total, (mu_I1, var_I1), (mu_I2, var_I2), (mu_I12, var_I12)

In [27]:
tiempo, I1, I2, I12 = evaluar_antitetico(n, repeticiones)

In [28]:
print("   Estimador        Media       Varianza")
print("===============================================")
print(f"  theta1(r)     {I1[0]:.6f}   {I1[1]:.6f}")
print(f"  theta1(1-r)   {I2[0]:.6f}   {I2[1]:.6f}")
print(f"  Antitético    {I12[0]:.6f}   {I12[1]:.6f}")
print("===============================================")
print(f"Tiempo total: {tiempo:.4f}")

   Estimador        Media       Varianza
  theta1(r)     0.632154   0.000142
  theta1(1-r)   0.627843   0.000116
  Antitético    0.629998   0.000012
Tiempo total: 0.0860


##**Método Acierto y error**

In [29]:
def MC (f, a, b, N):
  I1, I2 = 0, 0
  for i in range (n):
    r = random()
    I1 += f(r*(b-a)+a)
    I2 += f((1-r)*(b-a)+a)
  return I1/N, I2/N

In [30]:
starttime=time.perf_counter()
I1, I2, I12 = [], [], []
for j in range (1, 1000):
  i1, i2 = MC(g, 0, 1, 1000)
  I1. append(i1)
  I2. append(i2)
  I12.append((i1+i2)/2)
endtime = time.perf_counter()
t1 = endtime-starttime

iimu, iis2, iis, t2 = crudo(1000,1000)

NameError: name 'crudo' is not defined

In [ ]:
mu_theta_1 = np.mean(I1)
s2_theta_1 = np.var(I1)

mu_theta_1 = np.mean(I2)
s2_theta_1 = np.var(I2)

mu_theta_1 = np.mean(I12)
s2_theta_1 = np.var(I12)

In [ ]:
def crudo(n):
  G=[]
  for i in range (n):
    u=rd.random()
    G.append(g(u))
  return np.mean(G), np.var(G), np.std(G), G